# Índice<a id="ind"></a>

[**Introdução**](#limp_ini)

[**Etapa 1**](#etapa1)
* [Análise de uma amostra: identificação tipos mínimos e transformações](#ana_amo)
    - [Identificação das transformações possíveis](#ident_trans)
    - [Identificação dos subtipos mínimos](#obt_tipo_min)       
        
[**Etapa 2**](#etapa2)
* [Carregamento e tratamento dos dados totais](#carr_tot)
* [Comparação entre dados com e sem optimização](#comp)

In [13]:
# instala o pacote "optimizacaoDF"
!pip install git+https://github.com/joaogambaro/optimizacao_dataframes

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/joaogambaro/optimizacao_dataframes to /tmp/pip-req-build-otbylz0j
  Running command git clone --filter=blob:none --quiet https://github.com/joaogambaro/optimizacao_dataframes /tmp/pip-req-build-otbylz0j
  Resolved https://github.com/joaogambaro/optimizacao_dataframes to commit 4aa7233977ba0e6952eee6480f5331c670cd4e68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for optimizacao-dataframe: filename=optimizacao_dataframe-1.0.0-py3-none-any.whl size=6635 sha256=396ca47a0b62f690ae9997406fbc3014dbaf3f75c4507fb741f820ad443cb974
  Stored in directory: /tmp/pip-ephem-wheel-cache-unu46hl0/wheels/de/4f/e0/3ff529f8aca2eac6d2e3354bc7b2647f27e75aea8c75ac1ec2
Successfully built optimizacao-dataframe
  Attempting uninstall: optimizacao-dataframe
    Found existing installation: optimizacao-da

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import optimizacaoDF.coleta_amostra as amo
import optimizacaoDF.analise_resultados as ana
import optimizacaoDF.optimizacao_dataframe as opt

import os
import importlib
import warnings

pd.pandas.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# Introdução<a id="limp_ini"></a>
[Índice](#ind)

O objetivo central deste notebook foi desenvolver uma rotina para abrir um arquivo ".csv" com um   consumo reduzido de memória.  A redução foi obtida identificando as colunas de dados que pode ser escritas com subtipos de variáveis menores.

    Exemplo: para os dados abaixo, a coluna "idade" foi carregada com o tipo `float64`, "nome" com o tipo `object` e "peso" com o tipo `float64`. Contudo, estes tipos necessitam de mais memória do que o necessário para o armazenamento dos dados em questão. Para estes dados,  por exemplo, poderíamos escrever as colunas com os tipo, int8 para "idade", "classify" para "nome" e float32 para "peso" sem perda de informação, reduzindo o uso de memória.

|idade(float64)|nome(object) | peso(float64)|
|--|--|--|
|34.0| João|64.5 |
|32.0| Camila|75.8 |
|31.0| José|65.3 |
|26.0| Ana|58.0 |
    
Este trabalho foi dividido em etapas

**Etapa 1: analisar um pequena amostra dos dados**

Nesta etapa é carregado em um dataframe com uma amostra dos dados completos. O dataframe contém somente algumas linhas do arquivo original, que podem ser escolhidas aleatoriamente ou em sequência. A amostra selecionada é usada para identificar os subtipos mínimos de cada coluna e as transformações que podem ser feitas.

**Etapa 2: carregamento dos dados totais**

Nesta etapa os dados são carregados utilizando o subtipos mínimos identificados na etapa 1. Depois do carregamento, as transformações identificadas na etapa 1 são aplicadas e, por final, uma nova identificação de subtipos é feita. Estes últimos passos  podem reduzir ainda mais o tamanho do dataframe  na memória. O resultado final é salvo em um novo arquivo.


**Observações**
- para auxiliar nos processo das etapa 1 e 2 foi criado o pacote "optimizacaoDF" que foi instalado no incio do notebook

- Quando os dados são salvos no final do processo, os subtipos que foram definidos não são salvos no arquivo. Ou seja, quando os dados forem abertos outras vezes, devemos novamente informar os tipos para reduzir o consumo de memória. Ainda assim, é importante salvar os o resultado final em um arquivo, pois muitos arquivos grandes possuem linhas em branco que serão eliminadas com o salvamento do resultado final. Além disso, as demais transformações feitas nos dados ficarão salvas com o novo arquivo.




# Etapa 1<a id="etapa1"></a>
[Índice](#ind)

## Análise de uma amostra: identificação tipos mínimos e transformações<a id="ana_amo"></a>


A partir da amostra dos dados foram selecionadas algumas colunas de interesse, cujos nomes estão na lista `col_sel`. Para as colunas selecionadas são feitas as seguinte análises:


- Identificação das transformações possíveis:

Neste processo são identificadas algumas transformações possíveis. Como um exemplo, temos a coluna "price" que contém strings com valores de preço, (como "$500.00") que pode ter seus valores transformados para números.  A identificação é feita analisando individualmente cada coluna, e as funções das transformações são organizadas em um dicionário.
    

- Identificação dos subtipos mínimos: 

Este processo é feito usando o método `obtem_subtipos` do pacote `optimizacaoDF`, que retorna um dicionário com os nomes das colunas e os subtipos mínimos.


In [2]:
# caminho do arquivo com dados
current_path = os.getcwd()
file_name=Path(current_path)/"dados_originais/1_listings.csv"

# coleta uma amostra(primeiras linhas)
tamAm=500
df=amo.coleta_amostr(tamAm, file_name, is_rand_sample=False)

print("df.shape: ",df.shape)
display(df.head(3))

df.shape:  (500, 75)


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,53344884,https://www.airbnb.com/rooms/53344884,20220921172238,2022-09-22,city scrape,Suntuoso apartamento em Copacabana posto 6,suntuoso apartamento de frente para o mar <br ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,431412286,https://www.airbnb.com/users/show/431412286,Carlos Henrique,2021-11-11,NaN,NaN,within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/f4bec...,https://a0.muscache.com/im/pictures/user/f4bec...,NaN,10,10,"['email', 'phone']",t,t,NaN,Copacabana,NaN,-22.98299,-43.18904,Entire condo,Entire home/apt,6,NaN,4.5 baths,3.0,5.0,"[""Air conditioning"", ""Washer"", ""Security camer...","$3,500.00",3,365,3,3,365,365,3.0,365.0,NaN,t,30,60,90,365,2022-09-22,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,10,8,2,0,NaN
1,7801456,https://www.airbnb.com/rooms/7801456,20220921172238,2022-09-22,city scrape,Ipanema Vieira Souto - Linda Vista,Flat luxuoso com a melhor localização do Rio d...,"Ipanema é considerado o berço do rio, cheio de...",https://a0.muscache.com/pictures/99276394/dc9c...,40650139,https://www.airbnb.com/users/show/40650139,Carlos Alberto,2015-08-05,"Rio de Janeiro, Brazil","22 anos, Brasileiro, Solteiro, residencia fixa...",a few days or more,13%,100%,f,https://a0.muscache.com/im/users/40650139/prof...,https://a0.muscache.com/im/users/40650139/prof...,Ipanema,6,6,"['email', 'phone']",t,t,"Rio de Janeiro, Brazil",Ipanema,NaN,-22.98723,-43.20452,Entire serviced apartment,Entire home/apt,4,NaN,2 baths,2.0,3.0,"[""Cable TV"", ""Air conditioning"", ""Fire extingu...","$5,000.00",2,900,2,2,900,900,2.0,900.0,NaN,t,30,60,90,365,2022-09-22,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,5,5,0,0,NaN
2,14333905,https://www.airbnb.com/rooms/14333905,20220921172238,2022-09-22,city scrape,Casa compartilhada,"A 15 min. do complexo esportivo de Deodoro, Ca...",NaN,https://a0.muscache.com/pictures/15235501-16c4...,87749071,https://www.airbnb.com/users/show/87749071,Lindenberg,2016-08-03,"Rio de Janeiro, Brazil",NaN,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/409ec...,https://a0.muscache.com/im/pictures/user/409ec...,NaN,1,1,"['email', 'phone']",t,f,NaN,Pavuna,NaN,-22.80869,-43.38642,Entire home,Entire home/apt,12,NaN,1 bath,2.0,4.0,"[""Cable TV"", ""Air conditioning"", ""Hair dryer"",...",$681.00,1,1125,1,1,1125,1125,1.0,1125.0,NaN,t,30,60,90,365,2022-09-22,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,1,1,0,0,NaN


In [3]:
# colunas selecionadas
col_sel=['id',
         'host_id',
         'host_since',
         'host_location',
         'host_response_time',
         'host_response_rate',
         'host_acceptance_rate',
         'host_is_superhost',
         'host_verifications',
         'host_has_profile_pic',
         'host_identity_verified',
         'neighbourhood_cleansed',
         'latitude',
         'longitude',
         'room_type',
         'accommodates',
         'bathrooms_text',
         'bedrooms',
         'beds',
         'price',
         'minimum_nights',
         'maximum_nights',
         'number_of_reviews',
         'first_review',
         'last_review',
         'review_scores_rating',
         'review_scores_accuracy',
         'review_scores_cleanliness',
         'review_scores_checkin',
         'review_scores_communication',
         'review_scores_location',
         'review_scores_value',
         'calculated_host_listings_count',
         'calculated_host_listings_count_entire_homes',
         'calculated_host_listings_count_private_rooms',
         'calculated_host_listings_count_shared_rooms',
         'reviews_per_month']



### Identificação das transformações possíveis <a id="ident_trans"></a>
Colunas com transformações:


**host_response_rate:** os dados originais são strings com porcentagens (exemplo: "%50"). Na transformação, as strings foram transformadas em números decimais (para o exemplo temos 0.5).

**host_acceptance_rate:** mesma transformação de "host_response_rate".

**host_is_superhost:** os dados originais são strigns com 'f' e 't' que indicam False ou True. Na transformação, 'f' e 't' viram respectivamente os inteiros 0 e 1.

**host_has_profile_pic:** mesma transformação de "host_is_superhost" .

**host_identity_verified:** mesma transformação de "host_is_superhost".

**price:**  os dados originais são strings com preços ( exemplo: "$5,000.00"). Na transformação, as strings foram transformadas em números floats (para exemplo temos 5000.00).


In [4]:
# dicionário com as tansformações
dict_transf={    
    'host_response_rate':
        lambda x: float(x.removesuffix("%"))/100\
                if not pd.isna(x) and x.removesuffix("%").isdigit()\
                else float('NaN'),
    'host_acceptance_rate':
        lambda x: float(x.removesuffix("%"))/100\
                if not pd.isna(x) and x.removesuffix("%").isdigit()\
                else float('NaN'),
    'host_is_superhost':
        lambda x: 0 if x=='f' 
                    else ( 1 if x=='t' else float('NaN')),
    'host_has_profile_pic':
        lambda x: 0 if x=='f' 
                    else ( 1 if x=='t' else float('NaN')),
    'host_identity_verified':
        lambda x: 0 if x=='f' 
                    else ( 1 if x=='t' else float('NaN')),
    'price':
         lambda x: float(x.replace("$","").removesuffix(".00").replace(",",""))\
                  if not pd.isna(x)\
                  and x.replace("$","")\
                       .removesuffix(".00")\
                       .replace(",","").isdigit()\
                  else float('NaN')
     }

### Identificação dos subtipos mínimos<a id="obt_tipo_min"></a>
>**Observação:** em `obtem_subtipos` foi usado `"can_float_be_int=False"` para que colunas com float nunca sejam identificadas como int (se fosse usado True o número 23.00, por exemplo, poderia ser identificado como o int, 23). Como estamos lidando com uma amostra, a melhor opção é usar False para este parâmetro. Isto porque com True o método pode associar uma coluna float como int para a amostra, enquanto que nos dados totais pode ser que isto seja impossível. Este fato pode acontecer, por exemplo, quando quando na amostra não existem dados Nan (que são float), mas nos dados totais existem.

In [5]:
# seleciona os dados
df = df[col_sel]

# obtem os subtipos minimos
dict_sub = opt.obtem_subtipos(df, can_float_be_int=False)
dict_sub


{'id': 'uint64',
 'host_id': 'uint32',
 'host_since': 'object',
 'host_location': 'category',
 'host_response_time': 'category',
 'host_response_rate': 'category',
 'host_acceptance_rate': 'category',
 'host_is_superhost': 'category',
 'host_verifications': 'category',
 'host_has_profile_pic': 'category',
 'host_identity_verified': 'category',
 'neighbourhood_cleansed': 'category',
 'latitude': 'float32',
 'longitude': 'float32',
 'room_type': 'category',
 'accommodates': 'uint8',
 'bathrooms_text': 'category',
 'bedrooms': 'float32',
 'beds': 'float32',
 'price': 'object',
 'minimum_nights': 'uint16',
 'maximum_nights': 'uint16',
 'number_of_reviews': 'uint16',
 'first_review': 'object',
 'last_review': 'category',
 'review_scores_rating': 'float32',
 'review_scores_accuracy': 'float32',
 'review_scores_cleanliness': 'float32',
 'review_scores_checkin': 'float32',
 'review_scores_communication': 'float32',
 'review_scores_location': 'float32',
 'review_scores_value': 'float32',
 'calc

# Etapa 2<a id="etapa2"></a>
[Índice](#ind)

## Carregamento e tratamento dos dados totais<a id="carr_tot"></a>

Rotina seguida:
- 1. Carrega os dados com os subtipos mínimos identificados com a amostra

- 2. Aplica as transformações identificadas na amostra

- 3. Obtém novamente os subtipos mínimos para os dados

>**Observação:** Nesta etapa podemos encontrar novos subtipos mínimos, o que reduz ainda mais a memória usada pelo dataframe. Aqui, usamos o método `obtem_subtipos` com o parâmetro `"can_float_be_int=True"`, ou seja, agora tornamos possível números float serem identificados com int em situações adequadas. Ista escolha é feita porque o  método é aplicado nos dados completos e não em uma amostra.  Os novos subtipos que podem ocorrem devido a escolha do parâmetros 'can_float_be_int=True' e também devido aos dados transformados na etapa 2.

- 4. transforma os dados para os subtipos mínimos  da etapa 3 e salva os dados em um arquivo.


In [6]:
%%time

# 1. carrega os dados com os subtipos minimos
df_total=pd.read_csv(file_name,
                     low_memory=False,
                     usecols = col_sel, #colunas selecionadas
                     dtype = dict_sub   #dicionário com tranformações
                    )


# uso de memória
mem_df_total = ana.uso_memoria(df_total)
print(f'memória do df_total: {round(mem_df_total,2)}Mb')


memória do df_total: 7.13Mb
CPU times: user 723 ms, sys: 229 ms, total: 952 ms
Wall time: 953 ms


In [7]:
%%time

# 2. aplica as transformações
opt.object_to_float(df_total,dict_transf,inplace=True)


# uso de memória
mem_df_total = ana.uso_memoria(df_total)
print(f'memória do df_total: {round(mem_df_total,2)}Mb')

memória do df_total: 6.57Mb
CPU times: user 74.2 ms, sys: 328 µs, total: 74.5 ms
Wall time: 142 ms


In [8]:
%%time

# 3. pesquisa novamente os subtipos minimos
dict_sub = opt.obtem_subtipos(df_total, can_float_be_int=True)
dict_sub

CPU times: user 189 ms, sys: 0 ns, total: 189 ms
Wall time: 188 ms


{'host_since': 'category',
 'host_response_rate': 'float32',
 'host_acceptance_rate': 'float32',
 'host_is_superhost': 'float32',
 'host_has_profile_pic': 'float32',
 'host_identity_verified': 'float32',
 'price': 'uint32',
 'first_review': 'category'}

In [10]:
# 4. transforma os dados para os subtipos mínimos 
opt.tranforma_tipos(df_total,dict_sub, inplace=True)

# uso de memória
mem_df_total = ana.uso_memoria(df_total)
print(f'memória do df_total: {round(mem_df_total,2)}Mb')

memória do df_total: 3.56Mb


In [11]:
# salva o resultado final
current_path = os.getcwd()
path_root=Path(current_path).parent
df_total.to_csv(path_root/"dados/1_listings_opt.csv", index=False)

# Comparação entre dados com e sem optimização<a id="comp"></a>
[Índice](#ind)

Para finalizar, demostro a eficácia do método usado. Com este objetivo foram carregados os dados completos sem a identificação dos subtipos e foi medido o consumo de memória para os dados com e sem o tratamento proposto. Fazendo isto, podemos verificar que há uma redução de 87.46% na alocação de memória com o tratamento proposto. 

Também é mostrada uma comparação entre os tipos para os dados originais e para os dados com o tratamento. Esta comparação é feita pelo método `resultados_em_df` do pacote `optimizacaoDF`

>**Observação:** a ideia do tratamento é não precisar carregar os dados completos com o seu consumo máximo de memória. Para isto, utilizamos uma amostra para identificar os subtipos usados no carregamento dos dados totais. Nesta seção, os dados totais foram carregados na sua forma menos eficiente somente para mostrar a eficácia dos métodos aplicados, mas ideia central é não precisar fazer este carregamento.

In [12]:
# le o arquivo original(sem tranformação e sem conversao de tipo)
df_original = pd.read_csv(file_name,
                          low_memory=False,
                          usecols=col_sel
                         )


# redução de momória
porc = ana.porc_reducao(df_original,df_total)*100
mem_df_original = ana.uso_memoria(df_original)
mem_df_total = ana.uso_memoria(df_total)

print(f'memória do df_original: {round(mem_df_original,2)}Mb')
print(f'memória do df_total: {round(mem_df_total,2)}Mb')
print(f'Porcentagem na redução de memória: {round(porc,2)}%')

# compara com o resultado da optimização
ana.resultados_em_df(df_original, df_total)

memória do df_original: 28.4Mb
memória do df_total: 3.56Mb
Porcentagem na redução de memória: 87.46%


,sem_mud,com_mud
id,int64,uint64
host_id,int64,uint32
host_since,object,category
host_location,object,category
host_response_time,object,category
host_response_rate,object,float32
host_acceptance_rate,object,float32
host_is_superhost,object,float32
host_verifications,object,category
host_has_profile_pic,object,float32
